In [5]:
import pandas as pd

In [9]:
# The file is 1.63GB, which is too big for Github
# Save the file locally and read in using the file's path

path = 'C:/Users/puri.rudick/Downloads/all_train.csv.gz'

df = pd.read_csv(path, compression='gzip', error_bad_lines=False)

C:\Users\puri.rudick\AppData\Local\Temp\ipykernel_3432\1603302446.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(path, compression='gzip', error_bad_lines=False)


In [11]:
df.shape

(7000000, 29)

In [12]:
# Count missing values
df_na = pd.DataFrame(df.isna().sum())
df_na.rename(columns={df_na.columns[0]: 'count'}, inplace = True)

df_na = df_na[df_na['count'] != 0]
df_na['percent_missing'] = df_na['count']/df.shape[0]
df_na = df_na.sort_values('count', ascending=False)

df_na

,count,percent_missing


In [13]:
# Check for duplicate rows
print('Duplicate rows: ', df.duplicated(keep='first').sum())

Duplicate rows:  0
